In [24]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [46]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained('gpt2')

def load_dataset(tokenizer, file_path, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

dataset = load_dataset(tokenizer, 'queries.txt')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

C:\Users\Chand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
100%|██████████| 9/9 [00:40<00:00,  4.47s/it]


{'train_runtime': 40.2536, 'train_samples_per_second': 0.82, 'train_steps_per_second': 0.224, 'train_loss': 4.269185384114583, 'epoch': 3.0}


('./fine_tuned_gpt2\\tokenizer_config.json',
 './fine_tuned_gpt2\\special_tokens_map.json',
 './fine_tuned_gpt2\\vocab.json',
 './fine_tuned_gpt2\\merges.txt',
 './fine_tuned_gpt2\\added_tokens.json')

In [47]:
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

def generate_response(model, tokenizer, query):
    inputs = tokenizer.encode(query, return_tensors='pt', max_length=150, truncation=True)
    attention_mask = inputs.ne(tokenizer.pad_token_id).long()  # Create attention mask
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1, attention_mask=attention_mask)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
# Test query
test_query = "How do I know if I am getting a refund?"

# Generate and print the response
response = generate_response(fine_tuned_model, fine_tuned_tokenizer, test_query)
print(f"Query: {test_query}\nResponse: {response}")